# Deflection Profile Parametric Study for El Ghadioui2020
El Ghadioui2020_Structural behaviour of CFRP reinforced concrete members under bending and shear loads

In [ ]:
%matplotlib widget
# %matplotlib qt

In [ ]:
from bmcs_cross_section.api import MKappa, TShape, EC2, BarLayer, ReinfLayer
from bmcs_beam.api import BoundaryConditions, BoundaryConfig, DeflectionProfile, BeamDesign, LoadDeflectionParamsStudy
from os.path import join
import numpy as np
import pandas as pd
import sympy as sp
import matplotlib.pyplot as plt
import bmcs_utils.api as bu

In [ ]:
# Beam B-M-C-K1 - CFRP Bars
excel_file = join('../verification_data/el_ghadioui2020', 'fig_3.xlsx')
data = pd.read_excel(excel_file, engine='openpyxl')
ver_data = [data['B-M-C-K1_deflection'], data['B-M-C-K1_load']]

In [ ]:
# Beam B-M-C-K1 - CFRP Bars

# C 50/60
def get_dp(f_ck = None, E=None, low_kappa = None, high_kappa = None, n_kappa=None, n_load_steps=None, eps_tu=None):
    if eps_tu is None:
        eps_tu = 0.00008
    
    if n_load_steps is None:
        n_load_steps = 31
        
    if E is None:
        E = 33525
    
    if high_kappa is None:
        high_kappa = 0.00011
        
    if n_kappa is None:
        n_kappa = 200
        
    if low_kappa is None:
        low_kappa = 0
        
    mc = MKappa(low_kappa = low_kappa, high_kappa = high_kappa, n_kappa = n_kappa)

    if f_ck:
        f_ck = 90

        E = EC2.get_E_cm(f_ck)

        fctk_0_05 = EC2.get_f_ctk_0_05(f_ck)
        f_ctd = fctk_0_05
        # f_ctd = 0.85 * fctk_0_05 / 1.5
        f_ctd_fl = max((1.6 - 200 /1000) * f_ctd, f_ctd)

        mc.cs_design.matrix_.trait_set(
        E_cc=E,
        E_ct=E,
        eps_cr = f_ctd_fl/E,
        eps_tu = 2* f_ctd_fl/E,
        eps_cy = EC2.get_eps_c3(f_ck),
        eps_cu = EC2.get_eps_cu3(f_ck),
        mu=0.0,
        factor = 1
        )
    else:

        # mc.cs_design.matrix='EC2 with plateau'
        mc.cs_design.matrix_.trait_set(
                E_cc=E,
                E_ct=E,
                eps_cr=0.00004, #0.000125 * f_ck/30, # f_ctm /E,
                eps_cy=0.00175,
                eps_cu=0.0035,
                eps_tu=eps_tu,
                mu=0.0,
        #     f_cd = 30, # f_ck * 0.85/1.5,
            factor = 1
            )
    mc.cross_section_shape_.B = 400
    mc.cross_section_shape_.H = 200

    bl1 = ReinfLayer(name='carbon', z=35, A=2 * 70.14, matmod='carbon')
    bl1.matmod_.trait_set(E=135121, f_t=1891, factor = 1.)
    mc.cross_section_layout.add_layer(bl1)

    dp = DeflectionProfile(mc=mc, n_load_steps = n_load_steps)

    dp.add_fw_exp(ver_data[1], ver_data[0])

    dp.beam_design.beam_conf_name = BoundaryConfig.FOUR_PB
    dp.beam_design.beam_conf_name.first_load_distance = 1925
    dp.beam_design.L = 4700
    # dp.beam_design.L = 5000 # This gives perfect results
    dp.F_scale = 2/1000
    return dp

get_dp().interact()

In [ ]:
dp = get_dp()
empty = dp.mc.kappa_t[np.where(dp.mc.kappa_t < 0)]

In [ ]:
dp.mc.kappa_t

In [ ]:
dp = get_dp()
kappa_pos = dp.mc.kappa_t[np.where(dp.mc.kappa_t >= 0)]
kappa_pos

In [ ]:
emp = []
np.concatenate([emp, kappa_pos])

In [ ]:
# # Test f_ck
# """ [
#     {'f_ck': 30, 'low_kappa':0 , 'high_kappa':0.00012 },
#     {'f_ck': 50, 'low_kappa':0 , 'high_kappa':0.00012 },
#     {'f_ck': 70, 'low_kappa':-0.00001 , 'high_kappa':0.00011},
#     {'f_ck': 90, 'low_kappa':-0.00002 , 'high_kappa':0.00011},
#     ] """

# f_cks = [30, 50, 70, 90]
# dps = []
# dp_container = DP()
# for f_ck in f_cks:
#     dps.append(get_dp(f_ck=f_ck).interact())

# params_config = [['dp',   dp_container, dps],]
    
# LoadDeflectionParamsStudy(dp).run(params_config, ver_data)
    

# _, ax = plt.subplots()
# f, w = dp.get_Fw()
# ax.plot(w, f*2/1000)
# ax.plot(ver_data[0], ver_data[1])

In [ ]:
# Test E (concrete)
E_list = np.array([30000, 35000, 40000, 45000])

dp = get_dp()

# following fix is just to keep the sig, and not eps the same when changing E in MULTI-LINEAR Model
E = dp.mc.cs_design.matrix_.E_cc
eps_cy = dp.mc.cs_design.matrix_.eps_cy
eps_cr= dp.mc.cs_design.matrix_.eps_cr
sig_cy = E * eps_cy
sig_cr = E * eps_cr

sig_cy_list = np.full_like(E_list, sig_cy)
sig_cr_list = np.full_like(E_list, sig_cr)

params_config = [
    [['E_cc', dp.mc.cs_design.matrix_, E_list], 
     ['E_ct', dp.mc.cs_design.matrix_, E_list]],
]
# params_config = [
#     [['E_cc', dp.mc.cs_design.matrix_, E_list], 
#      ['E_ct', dp.mc.cs_design.matrix_, E_list], 
#      ['eps_cy', dp.mc.cs_design.matrix_, sig_cy_list / E_list], 
#      ['eps_cr', dp.mc.cs_design.matrix_, sig_cr_list / E_list]],
# ]
LoadDeflectionParamsStudy(dp).run(params_config, ver_data)

In [ ]:
# Test E_carbon
E_carbon = [130000, 170000, 210000, 250000]

dp = get_dp(n_kappa = 500)
E_cc = dp.mc.cs_design.matrix_.E_cc
E_list = [E_cc, E_cc + 0.01, E_cc, E_cc + 0.01] # Just to update state because changing E_carbon doesn't update the state automatically
params_config = [
    [['E', dp.mc.cross_section_layout.items[0].matmod_, E_carbon], 
     ['E_cc', dp.mc.cs_design.matrix_, E_list]]
]
LoadDeflectionParamsStudy(dp).run(params_config, ver_data)

In [ ]:
# Test carbon strength
f_t_list = [1800, 2200, 2600, 3000]
dp = get_dp(high_kappa = 0.00018)
E_cc = dp.mc.cs_design.matrix_.E_cc
E_list = [E_cc, E_cc + 0.01, E_cc, E_cc + 0.01] # Just to update state because changing E_carbon doesn't update the state automatically
params_config = [
    [['f_t', dp.mc.cross_section_layout.items[0].matmod_, f_t_list], 
     ['E_cc', dp.mc.cs_design.matrix_, E_list]]
]
LoadDeflectionParamsStudy(dp).run(params_config, ver_data)

In [ ]:
# Test concrete compressive strength
f_cm_list = np.array([30, 50, 70, 90])

dp = get_dp(n_kappa=500, high_kappa=0.00013)

E = dp.mc.cs_design.matrix_.E_cc
E_list = np.full_like(f_cm_list, E)

params_config = [
     ['eps_cy', dp.mc.cs_design.matrix_, f_cm_list / E_list],
]
LoadDeflectionParamsStudy(dp).run(params_config, ver_data)

In [ ]:
# Test concrete tensile strength
f_ctd_list = np.array([0, 2.8, 4, 4.5, 5])

dp = get_dp(n_kappa=500, high_kappa=0.00013, eps_tu = 2*0.00008, n_load_steps=250)

E = dp.mc.cs_design.matrix_.E_cc
E_list = np.full_like(f_ctd_list, E)

params_config = [
     ['eps_cr', dp.mc.cs_design.matrix_, f_ctd_list / E_list],
]
LoadDeflectionParamsStudy(dp).run(params_config, ver_data)